## Paquets i dades originals ## 

In [ ]:
import pandas as pd


taula = pd.read_pickle('./data/processed/dataframe.pkl')
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

Prèviament, he tractat les dades creant noves columnes que classifiquen millor tota la informació disponible

In [ ]:
taula.head()

Distribuiré les dades en 3 grups (més un grup que conté la totalitat de dades):
- **AMB_PA**: aquest grup està compost per pacients que contenen el codi ICD de pneumònia aspirativa al seu historial.
- **AMB_PA_MECVV**: aquest grup conté els pacients que compleixen les següents característiques: 
    - Han donat positiu al MECVV (és a dir, que tenen disfàgia i una alteració en la seguretat o en l'eficàcia)
    - Han sigut diagnosticats amb el codi ICD de pneumònia
    - Han passat menys de 30 dies entre el diagnòstic amb l'ICD de pneumònia i el resultat positiu al MECVV
- **SENSE_PA**: aquest grup està format per pacients que contenen el codi ICD de pneumònia o bé:
    - Contenen el codi ICD de pneumònia
    - Han donat positiu al MECVV 
    - Han passat més de 30 dies entre el diagnòstic amb l'ICD de pneumònia i  el resultat positiu del MECVV 
- **TOTAL**: aquest grup conté tots els pacients de l'estudi *#TODO: hacer descriptiva de este grupo*

# AMB_PA 

In [ ]:
AMB_PA = taula[taula["PA diagnosticada"] == 1.0]
AMB_PA.head()

# AMB_PA_MECVV

In [ ]:
AMB_PA_MECVV = taula[
    (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] < 30)] 
AMB_PA_MECVV.head()

# SENSE_PA

In [ ]:
SENSE_PA = taula[
           (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] > 30) & 
           (taula['P diagnosticada'] == 1.0)]
SENSE_PA.head()

# TOTAL

In [ ]:
TOTAL = taula
taula.head()

Ara que tinc totes les dades classificades en aquests grups, faré una descriptiva. També compararé els diferents grups amb el pvalor mitjançant T-test, test de Mann-Whitney o test Xi-quadrat per mirar la seva distribució, la qual esperaré que sigui similar entre el grup AMB_PA i AMB_PA_MECVV, i diferent entre SENSE_PA i els altres 2 grups. Els criteris que seguiré per utilitzar un test o un altre és, primer, fer un test de Shapiro-Wilk per comprovar si segueixen una distribució normal, si és així, usaré el T-test, si no, utilitzaré el de Mann-Whitney. Finalment, si la columna conté valors categòrics com és el cas del sexe (F/M), faré servir el test Xi-quadrat.


## Nombre de pacients

In [ ]:
num_AMB_PA = len(AMB_PA)
num_AMB_PA_MECVV= len(AMB_PA_MECVV)
num_SENSE_PA = len(SENSE_PA)
num_TOTAL = len(TOTAL)

print(num_AMB_PA, num_AMB_PA_MECVV, num_SENSE_PA, num_TOTAL)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
stat, p_value = shapiro(range(num_AMB_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(range(num_AMB_PA_MECVV))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(range(num_SENSE_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(range(num_TOTAL))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Tots 4 grups segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA.index, AMB_PA_MECVV.index, equal_var=False)
print(f"P-valor entre AMB_PA i AMB_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA.index, SENSE_PA.index, equal_var=False)
print(f"P-valor entre AMB_PA i SENSE_PA: {p_val_13:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV.index, SENSE_PA.index, equal_var=False)
print(f"P-valor entre AMB_PA_MECVV i SENSE_PA: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

In [ ]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA.index, TOTAL.index, equal_var=False)
print(f"P-valor entre AMB_PA i TOTAL: {p_val_14:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV.index, TOTAL.index, equal_var=False)
print(f"P-valor entre AMB_PA_MECVV i TOTAL: {p_val_24:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

In [ ]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA.index, TOTAL.index, equal_var=False)
print(f"P-valor entre SENSE_PA i TOTAL: {p_val_34:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

## Edat

In [ ]:
edat_AMB_PA = AMB_PA['edat'].mean()
desviacio_estandard_edat = AMB_PA['edat'].std()

resultat = f"{edat_AMB_PA:.2f} ± {desviacio_estandard_edat:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultat)


In [ ]:
edat_AMB_PA_MECVV = AMB_PA_MECVV['edat'].mean()
desviacio_estandard_edat = AMB_PA_MECVV['edat'].std()

resultat = f"{edat_AMB_PA_MECVV:.2f} ± {desviacio_estandard_edat:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultat)

In [ ]:
edat_SENSE_PA = SENSE_PA['edat'].mean()
desviacio_estandard_edat = SENSE_PA['edat'].std()

resultat = f"{edat_SENSE_PA:.2f} ± {desviacio_estandard_edat:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultat)

In [ ]:
edat_TOTAL = TOTAL['edat'].mean()
desviacio_estandard_TOTAL = TOTAL['edat'].std()

resultat = f"{edat_SENSE_PA:.2f} ± {desviacio_estandard_edat:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
stat, p_value = shapiro(AMB_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(AMB_PA_MECVV['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(SENSE_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(TOTAL['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Tots els grups, menys el quart (TOTAL), segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['edat'], AMB_PA_MECVV['edat'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
stat, p_value_13 = mannwhitneyu(AMB_PA['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

In [ ]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

In [ ]:
stat, p_value_14 = mannwhitneyu(AMB_PA['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

In [ ]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

In [ ]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

## Sexe

In [ ]:
sexe_AMB_PA = AMB_PA['sexe'].value_counts()
sexe_AMB_PA_MECVV = AMB_PA_MECVV['sexe'].value_counts()
sexe_SENSE_PA = SENSE_PA['sexe'].value_counts()
sexe_TOTAL = TOTAL['sexe'].value_counts()

In [ ]:
if 'F' in sexe_AMB_PA:
    percent_female1 = (sexe_AMB_PA['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA")

In [ ]:
if 'F' in sexe_AMB_PA_MECVV:
    percent_female2 = (sexe_AMB_PA_MECVV['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA_MECVV: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA_MECVV")

In [ ]:
if 'F' in sexe_SENSE_PA:
    percent_female3 = (sexe_SENSE_PA['F'] / len(SENSE_PA)) * 100
    print(f"Percentatge de sexe femení en SENSE_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en SENSE_PA")


In [ ]:
if 'F' in sexe_TOTAL:
    percent_female3 = (sexe_TOTAL['F'] / len(TOTAL)) * 100
    print(f"Percentatge de sexe femení en TOTAL: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en TOTAL")

#### Comparació distributiva

##### Normalitat i p-valor

Al ser una variable categòrica no té sentit comprovar la seva normalitat. Realitzaré directament el test Xi-quadrat

In [ ]:
contingency_table_AMB_PA_AMB_PA_MECVV = pd.crosstab(index=AMB_PA['sexe'], columns=AMB_PA_MECVV['sexe'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_AMB_PA_AMB_PA_MECVV)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_AMB_PA_SENSE_PA = pd.crosstab(index=AMB_PA['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_AMB_PA_MECVV_SENSE_PA = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_MECVV_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['sexe'], columns=TOTAL['sexe'])

chi2_AMB_PA_TOTAL, p_value_AMB_PA_TOTAL, dof_AMB_PA_TOTAL, expected_AMB_PA_TOTAL = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_AMB_PA_TOTAL)
print("P-valor entre AMB_PA i TOTAL:", p_value_AMB_PA_TOTAL)

if p_value_AMB_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=TOTAL['sexe'])

chi2_AMB_PA_MECVV_TOTAL, p_value_AMB_PA_MECVV_TOTAL, dof_AMB_PA_MECVV_TOTAL, expected_AMB_PA_MECVV_TOTAL = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_AMB_PA_MECVV_TOTAL)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_AMB_PA_MECVV_TOTAL)

if p_value_AMB_PA_MECVV_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['sexe'], columns=TOTAL['sexe'])

chi2_SENSE_PA_TOTAL, p_value_SENSE_PA_TOTAL, dof_SENSE_PA_TOTAL, expected_SENSE_PA_TOTAL = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_SENSE_PA_TOTAL)
print("P-valor entre SENSE_PA i TOTAL:", p_value_SENSE_PA_TOTAL)

if p_value_SENSE_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.")

## Ingressos (admissions)

In [ ]:
Mitjana_ingresos = AMB_PA['Nombre ingressos'].mean()
desviacio_estandard_ingresos = AMB_PA['Nombre ingressos'].std()

resultat = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultat)

In [ ]:
Mitjana_ingresos = AMB_PA_MECVV['Nombre ingressos'].mean()
desviacio_estandard_ingresos = AMB_PA_MECVV['Nombre ingressos'].std()

resultat = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultat)

In [ ]:
Mitjana_ingresos = SENSE_PA['Nombre ingressos'].mean()
desviacio_estandard_ingresos = SENSE_PA['Nombre ingressos'].std()

resultat = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultat)

In [ ]:
Mitjana_ingresos = TOTAL['Nombre ingressos'].mean()
desviacio_estandard_ingresos = TOTAL['Nombre ingressos'].std()

resultat = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
stat, p_value = shapiro(AMB_PA['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(AMB_PA_MECVV['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(SENSE_PA['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(TOTAL['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Només el segon (AMB_PA_MECVV) i tercer (SENSE_PA) grup segueixen una distribució normal

In [ ]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Nombre ingressos'], AMB_PA_MECVV['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

In [ ]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Nombre ingressos'], SENSE_PA['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

In [ ]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['Nombre ingressos'], SENSE_PA['Nombre ingressos'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

In [ ]:
stat, p_value_14 = mannwhitneyu(AMB_PA['Nombre ingressos'], TOTAL['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

In [ ]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Nombre ingressos'], TOTAL['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

In [ ]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['Nombre ingressos'], TOTAL['Nombre ingressos'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

## Disfàgia

In [ ]:
disf_AMB_PA= (AMB_PA['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA:.2f}%")

In [ ]:
disf_AMB_PA_MECVV = (AMB_PA_MECVV['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA_MECVV:.2f}%")

In [ ]:
disf_SENSE_PA= (SENSE_PA['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_SENSE_PA:.2f}%")

In [ ]:
disf_TOTAL= (TOTAL['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_TOTAL:.2f}%")

#### Comparació distributiva

##### Normalitat i p-valor

Al ser una variable categòrica no té sentit comprovar la seva normalitat. Realitzaré directament el test Xi-quadrat

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['DO diagnosticada'], columns=AMB_PA_MECVV['DO diagnosticada'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['DO diagnosticada'], columns=SENSE_PA['DO diagnosticada'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['DO diagnosticada'], columns=SENSE_PA['DO diagnosticada'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['DO diagnosticada'], columns=TOTAL['DO diagnosticada'])

chi2_AMB_PA_TOTAL, p_value_AMB_PA_TOTAL, dof_AMB_PA_TOTAL, expected_AMB_PA_TOTAL = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_AMB_PA_TOTAL)
print("P-valor entre AMB_PA i TOTAL:", p_value_AMB_PA_TOTAL)

if p_value_AMB_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['DO diagnosticada'], columns=TOTAL['DO diagnosticada'])

chi2_AMB_PA_MECVV_TOTAL, p_value_AMB_PA_MECVV_TOTAL, dof_AMB_PA_MECVV_TOTAL, expected_AMB_PA_MECVV_TOTAL = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_AMB_PA_MECVV_TOTAL)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_AMB_PA_MECVV_TOTAL)

if p_value_AMB_PA_MECVV_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['DO diagnosticada'], columns=TOTAL['DO diagnosticada'])

chi2_SENSE_PA_TOTAL, p_value_SENSE_PA_TOTAL, dof_SENSE_PA_TOTAL, expected_SENSE_PA_TOTAL = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_SENSE_PA_TOTAL)
print("P-valor entre SENSE_PA i TOTAL:", p_value_SENSE_PA_TOTAL)

if p_value_SENSE_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.")

## Index de Barthel

### Mitjana

In [ ]:
Mitjana_barthel = AMB_PA['Barthel resultats'].mean()
desviacio_estandard_barthel = AMB_PA['Barthel resultats'].std()

resultat = f"{Mitjana_barthel:.2f} ± {desviacio_estandard_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultat)

In [ ]:
Mitjana_barthel = AMB_PA_MECVV['Barthel resultats'].mean()
desviacio_estandard_barthel = AMB_PA_MECVV['Barthel resultats'].std()

resultat = f"{Mitjana_barthel:.2f} ± {desviacio_estandard_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultat)

In [ ]:
Mitjana_barthel = SENSE_PA['Barthel resultats'].mean()
desviacio_estandard_barthel = SENSE_PA['Barthel resultats'].std()

resultat = f"{Mitjana_barthel:.2f} ± {desviacio_estandard_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
stat, p_value = shapiro(AMB_PA['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(AMB_PA_MECVV['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(SENSE_PA['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(TOTAL['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Només el tercer grup (SENSE_PA) segueix una distribució normal

In [ ]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'], AMB_PA_MECVV['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

In [ ]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'], SENSE_PA['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

In [ ]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'], SENSE_PA['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_13)

In [ ]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'], TOTAL['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_value_13)

In [ ]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'], TOTAL['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

In [ ]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'], TOTAL['Barthel resultats'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

### Valors de dependència

##### AMB_PA

In [ ]:
indepe_AMB_PA = (AMB_PA['Barthel resultats'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA: {indepe_AMB_PA:.2f}%")

In [ ]:
mod_AMB_PA = (AMB_PA['Barthel resultats'].between(61, 95)).mean() * 100
print(f"Percentatge de mod_AMB_PA: {mod_AMB_PA:.2f}%")

In [ ]:
sev_AMB_PA = (AMB_PA['Barthel resultats'].between(21, 60)).mean() * 100
print(f"Percentatge de sev_AMB_PA: {sev_AMB_PA:.2f}%")

In [ ]:
total_AMB_PA = (AMB_PA['Barthel resultats'] <= 20.0 ).mean() * 100
print(f"Percentatge de total_AMB_PA: {total_AMB_PA:.2f}%")

##### AMB_PA_MECVV

In [ ]:
indepe_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA_MECVV: {indepe_AMB_PA_MECVV:.2f}%")

In [ ]:
mod_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'].between(61,95)).mean() * 100
print(f"Percentatge de mod_AMB_PA_MECVV: {mod_AMB_PA_MECVV:.2f}%")

In [ ]:
sev_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'].between(21,60)).mean() * 100
print(f"Percentatge de sev_AMB_PA_MECVV: {sev_AMB_PA_MECVV:.2f}%")

In [ ]:
total_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'] <= 20.0).mean() * 100
print(f"Percentatge de total_AMB_PA_MECVV: {total_AMB_PA_MECVV:.2f}%")

##### SENSE_PA

In [ ]:
indepe_SENSE_PA = (SENSE_PA['Barthel resultats'] == 100.0).mean() * 100
print(f"Percentatge de indepe_SENSE_PA: {indepe_SENSE_PA:.2f}%")

In [ ]:
mod_SENSE_PA = (SENSE_PA['Barthel resultats'].between(61,95)).mean() * 100
print(f"Percentatge de mod_SENSE_PA: {mod_SENSE_PA:.2f}%")

In [ ]:
sev_SENSE_PA = (SENSE_PA['Barthel resultats'].between(21,60)).mean() * 100
print(f"Percentatge de sev_SENSE_PA: {sev_SENSE_PA:.2f}%")

In [ ]:
total_SENSE_PA = (SENSE_PA['Barthel resultats'] <= 20).mean() * 100
print(f"Percentatge de total_SENSE_PA: {total_SENSE_PA:.2f}%")

#### P-valor

##### Independència

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Barthel resultats'] == 100.0, columns=AMB_PA_MECVV['Barthel resultats'] == 100.0)

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Barthel resultats'] == 100.0, columns=SENSE_PA['Barthel resultats'] == 100.0)

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Barthel resultats'] == 100.0, columns=SENSE_PA['Barthel resultats'] == 100.0)

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions Barthel en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Barthel resultats'] == 100.0, columns=TOTAL['Barthel resultats'] == 100.0)

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Barthel resultats'] == 100.0, columns=TOTAL['Barthel resultats'] == 100.0)

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Barthel resultats'] == 100.0, columns=TOTAL['Barthel resultats'] == 100.0)

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en SENSE_PA i TOTAL.")

##### Moderada

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Barthel resultats'].between(61, 95), columns=AMB_PA_MECVV['Barthel resultats'].between(61, 95))

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Barthel resultats'].between(61, 95), columns=SENSE_PA['Barthel resultats'].between(61, 95))

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Barthel resultats'].between(61, 95), columns=SENSE_PA['Barthel resultats'].between(61, 95))

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Barthel resultats'].between(61, 95), columns=TOTAL['Barthel resultats'].between(61, 95))

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Barthel resultats'].between(61, 95), columns=TOTAL['Barthel resultats'].between(61, 95))

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Barthel resultats'].between(61, 95), columns=TOTAL['Barthel resultats'].between(61, 95))

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en SENSE_PA i TOTAL.")

##### Severa

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Barthel resultats'].between(21,60), columns=AMB_PA_MECVV['Barthel resultats'].between(21,60))

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Barthel resultats'].between(21,60), columns=SENSE_PA['Barthel resultats'].between(21,60))

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Barthel resultats'].between(21,60), columns=SENSE_PA['Barthel resultats'].between(21,60))

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Barthel resultats'].between(21,60), columns=TOTAL['Barthel resultats'].between(21,60))

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Barthel resultats'].between(21,60), columns=TOTAL['Barthel resultats'].between(21,60))

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Barthel resultats'].between(21,60), columns=TOTAL['Barthel resultats'].between(21,60))

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en SENSE_PA i TOTAL.")

##### Total

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Barthel resultats'] <= 20.0, columns=AMB_PA_MECVV['Barthel resultats'] <= 20.0)

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Barthel resultats'] <= 20.0, columns=SENSE_PA['Barthel resultats'] <= 20.0)

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Barthel resultats'] <= 20.0, columns=SENSE_PA['Barthel resultats'] <= 20.0)

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Barthel resultats'] <= 20.0, columns=TOTAL['Barthel resultats'] <= 20.0)

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Barthel resultats'] <= 20.0, columns=TOTAL['Barthel resultats'] <= 20.0)

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Barthel resultats'] <= 20.0, columns=TOTAL['Barthel resultats'] <= 20.0)

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de Barthel en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de Barthel en SENSE_PA i TOTAL.")

## Index del MNA

Primer convertiré la columna a tipus numèric ja que els valors que conté són de tipus objecte i amb aquest tipus no puc calcular la mitjana ni la desviació estàndard.

In [ ]:
AMB_PA.loc[:, 'MNA resultats'] = pd.to_numeric(AMB_PA['MNA resultats'], errors='coerce')
AMB_PA_MECVV.loc[:, 'MNA resultats'] = pd.to_numeric(AMB_PA_MECVV['MNA resultats'], errors='coerce')
SENSE_PA.loc[:, 'MNA resultats'] = pd.to_numeric(SENSE_PA['MNA resultats'], errors='coerce')
TOTAL.loc[:, 'MNA resultats'] = pd.to_numeric(TOTAL['MNA resultats'], errors='coerce')

### Mitjana

In [ ]:
Mitjana_mna = AMB_PA['MNA resultats'].mean()
desviacio_estandard_mna = AMB_PA['MNA resultats'].std()

resultat = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultat)

In [ ]:
Mitjana_mna = AMB_PA_MECVV['MNA resultats'].mean()
desviacio_estandard_mna = AMB_PA_MECVV['MNA resultats'].std()

resultat = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultat)

In [ ]:
Mitjana_mna = SENSE_PA['MNA resultats'].mean()
desviacio_estandard_mna = SENSE_PA['MNA resultats'].std()

resultat = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultat)

In [ ]:
Mitjana_mna = TOTAL['MNA resultats'].mean()
desviacio_estandard_mna = TOTAL['MNA resultats'].std()

resultat = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
stat, p_value = shapiro(AMB_PA['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(AMB_PA_MECVV['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(SENSE_PA['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(TOTAL['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Tots els grups segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['MNA resultats'], AMB_PA_MECVV['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['MNA resultats'], SENSE_PA['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['MNA resultats'], SENSE_PA['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['MNA resultats'], TOTAL['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['MNA resultats'], TOTAL['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['MNA resultats'], TOTAL['MNA resultats'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Valors de malnutrició

##### AMB_PA

In [ ]:
norm_AMB_PA = AMB_PA['MNA resultats'].between(24, 30)
Percentatge_norm_AMB_PA = norm_AMB_PA.mean() * 100
num_observaciones_norm_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de norm_AMB_PA: {Percentatge_norm_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observaciones_norm_AMB_PA}")

In [ ]:
malnutr_AMB_PA = AMB_PA['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA = malnutr_AMB_PA.mean() * 100
num_observacions_malnutr_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de malnutr_AMB_PA: {Percentatge_malnutr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observacions_malnutr_AMB_PA}")

In [ ]:
malnut_AMB_PA = AMB_PA['MNA resultats'] < 17
Percentatge_malnut_AMB_PA = malnut_AMB_PA.mean() * 100
num_observacions_malnut_AMB_PA = malnut_AMB_PA.sum()

print(f"Percentatge de malnut_AMB_PA: {Percentatge_malnut_AMB_PA:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA: {num_observacions_malnut_AMB_PA}")

##### AMB_PA_MECVV

In [ ]:
norm_AMB_PA_MECVV = AMB_PA_MECVV['MNA resultats'].between(24, 30)
Percentatge_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.mean() * 100
num_observacions_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de norm_AMB_PA_MECVV: {Percentatge_norm_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observacions_norm_AMB_PA_MECVV}")

In [ ]:
malnutr_AMB_PA_MECVV = AMB_PA_MECVV['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA_MECVV = malnutr_AMB_PA_MECVV.mean() * 100
num_observacions_malnutr_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de malnutr_AMB_PA_MECVV: {Percentatge_malnutr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observacions_malnutr_AMB_PA_MECVV}")

In [ ]:
malnut_AMB_PA_MECVV = AMB_PA_MECVV['MNA resultats'] < 17
Percentatge_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.mean() * 100
num_observacions_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.sum()

print(f"Percentatge de malnut_AMB_PA_MECVV: {Percentatge_malnut_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA_MECVV: {num_observacions_malnut_AMB_PA_MECVV}")

##### SENSE_PA

In [ ]:
norm_SENSE_PA = SENSE_PA['MNA resultats'].between(24, 30)
Percentatge_norm_SENSE_PA = norm_SENSE_PA.mean() * 100
num_observacions_norm_SENSE_PA = norm_SENSE_PA.sum()

print(f"Percentatge de norm_SENSE_PA: {Percentatge_norm_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en norm_SENSE_PA: {num_observacions_norm_SENSE_PA}")

In [ ]:
malnutr_SENSE_PA = SENSE_PA['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_SENSE_PA = malnutr_SENSE_PA.mean() * 100
num_observacions_malnutr_SENSE_PA = malnutr_SENSE_PA.sum()

print(f"Percentatge de malnutr_SENSE_PA: {Percentatge_malnutr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnutr_SENSE_PA: {num_observacions_malnutr_SENSE_PA}")

In [ ]:
malnut_SENSE_PA = SENSE_PA['MNA resultats'] < 17
Percentatge_malnut_SENSE_PA = malnut_SENSE_PA.mean() * 100
num_observacions_malnut_SENSE_PA = malnut_SENSE_PA.sum()

print(f"Percentatge de malnut_SENSE_PA: {Percentatge_malnut_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnut_SENSE_PA: {num_observacions_malnut_SENSE_PA}")

##### TOTAL

In [ ]:
norm_TOTAL = TOTAL['MNA resultats'].between(24, 30)
Percentatge_norm_TOTAL = norm_TOTAL.mean() * 100
num_observacions_norm_TOTAL = norm_TOTAL.sum()

print(f"Percentatge de norm_TOTAL: {Percentatge_norm_TOTAL:.2f}%")
print(f"Nombre d'observacions en norm_TOTAL: {num_observacions_norm_TOTAL}")

In [ ]:
malnutr_TOTAL = TOTAL['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_TOTAL = malnutr_TOTAL.mean() * 100
num_observacions_malnutr_TOTAL = malnutr_TOTAL.sum()

print(f"Percentatge de malnutr_TOTAL: {Percentatge_malnutr_TOTAL:.2f}%")
print(f"Nombre d'observacions en malnutr_TOTAL: {num_observacions_malnutr_TOTAL}")

In [ ]:
malnut_TOTAL = TOTAL['MNA resultats'] < 17
Percentatge_malnut_TOTAL = malnut_TOTAL.mean() * 100
num_observacions_malnut_TOTAL = malnut_TOTAL.sum()

print(f"Percentatge de malnut_TOTAL: {Percentatge_malnut_TOTAL:.2f}%")
print(f"Nombre d'observacions en malnut_TOTAL: {num_observacions_malnut_TOTAL}")

#### P-valor

##### Normal

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['MNA resultats'].between(24, 30), columns=AMB_PA_MECVV['MNA resultats'].between(24, 30))

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['MNA resultats'].between(24, 30), columns=SENSE_PA['MNA resultats'].between(24, 30))

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['MNA resultats'].between(24, 30), columns=SENSE_PA['MNA resultats'].between(24, 30))

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['MNA resultats'].between(24, 30), columns=TOTAL['MNA resultats'].between(24, 30))

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['MNA resultats'].between(24, 30), columns=TOTAL['MNA resultats'].between(24, 30))

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['MNA resultats'].between(24, 30), columns=TOTAL['MNA resultats'].between(24, 30))

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en SENSE_PA i TOTAL.")

##### Risc de malnutrició

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['MNA resultats'].between(17, 23.5), columns=AMB_PA_MECVV['MNA resultats'].between(17, 23.5))

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['MNA resultats'].between(17, 23.5), columns=SENSE_PA['MNA resultats'].between(17, 23.5))

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['MNA resultats'].between(17, 23.5), columns=SENSE_PA['MNA resultats'].between(17, 23.5))

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['MNA resultats'].between(17, 23.5), columns=TOTAL['MNA resultats'].between(17, 23.5))

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['MNA resultats'].between(17, 23.5), columns=TOTAL['MNA resultats'].between(17, 23.5))

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['MNA resultats'].between(17, 23.5), columns=TOTAL['MNA resultats'].between(17, 23.5))

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en SENSE_PA i TOTAL.")

##### Malnutrició

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['MNA resultats'] < 17, columns=AMB_PA_MECVV['MNA resultats'] < 17)

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['MNA resultats'] < 17, columns=SENSE_PA['MNA resultats'] < 17)

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['MNA resultats'] < 17, columns=SENSE_PA['MNA resultats'] < 17)

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['MNA resultats'] < 17, columns=TOTAL['MNA resultats'] < 17)

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['MNA resultats'] < 17, columns=TOTAL['MNA resultats'] < 17)

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['MNA resultats'] < 17, columns=TOTAL['MNA resultats'] < 17)

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de MNA en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de MNA en SENSE_PA i TOTAL.")

## Index de l'EMINA

### Mitjana

In [ ]:
Mitjana_emina = AMB_PA['EMINA sumatoris comparats'].mean()
desviacio_estandard_emina = AMB_PA['EMINA sumatoris comparats'].std()

resultat = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultat)

In [ ]:
Mitjana_emina = AMB_PA_MECVV['EMINA sumatoris comparats'].mean()
desviacio_estandard_emina = AMB_PA_MECVV['EMINA sumatoris comparats'].std()

resultat = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultat)

In [ ]:
Mitjana_emina = SENSE_PA['EMINA sumatoris comparats'].mean()
desviacio_estandard_emina = SENSE_PA['EMINA sumatoris comparats'].std()

resultat = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultat)

In [ ]:
Mitjana_emina = TOTAL['EMINA sumatoris comparats'].mean()
desviacio_estandard_emina = TOTAL['EMINA sumatoris comparats'].std()

resultat = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
stat, p_value = shapiro(AMB_PA['EMINA sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(AMB_PA_MECVV['EMINA sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(SENSE_PA['EMINA sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(TOTAL['EMINA sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Tots els grups segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA sumatoris comparats'], AMB_PA_MECVV['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA sumatoris comparats'], SENSE_PA['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA sumatoris comparats'], SENSE_PA['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['EMINA sumatoris comparats'], TOTAL['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['EMINA sumatoris comparats'], TOTAL['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['EMINA sumatoris comparats'], TOTAL['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Valors EMINA

##### AMB_PA

In [ ]:
lowr_AMB_PA = AMB_PA['EMINA sumatoris comparats'].between(0, 5)
Percentatge_lowr_AMB_PA = lowr_AMB_PA.mean() * 100
num_observacions_lowr_AMB_PA = lowr_AMB_PA.sum()

print(f"Percentatge de lowr_AMB_PA: {Percentatge_lowr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA: {num_observacions_lowr_AMB_PA}")

In [ ]:
modr_AMB_PA = AMB_PA['EMINA sumatoris comparats'].between(6, 10)
Percentatge_modr_AMB_PA = modr_AMB_PA.mean() * 100
num_observacions_modr_AMB_PA = modr_AMB_PA.sum()

print(f"Percentatge de modr_AMB_PA: {Percentatge_modr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA: {num_observacions_modr_AMB_PA}")

In [ ]:
highr_AMB_PA = AMB_PA['EMINA sumatoris comparats'].between(11, 15)
Percentatge_highr_AMB_PA = highr_AMB_PA.mean() * 100
num_observacions_highr_AMB_PA = highr_AMB_PA.sum()

print(f"Percentatge de highr_AMB_PA: {Percentatge_highr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA: {num_observacions_highr_AMB_PA}")

##### AMB_PA_MECVV

In [ ]:
lowr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA sumatoris comparats'].between(0, 5)
Percentatge_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.mean() * 100
num_observacions_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.sum()

print(f"Percentatge de lowr_AMB_PA_MECVV: {Percentatge_lowr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA_MECVV: {num_observacions_lowr_AMB_PA_MECVV}")

In [ ]:
modr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA sumatoris comparats'].between(6, 10)
Percentatge_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.mean() * 100
num_observacions_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.sum()

print(f"Percentatge de modr_AMB_PA_MECVV: {Percentatge_modr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA_MECVV: {num_observacions_modr_AMB_PA_MECVV}")

In [ ]:
highr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA sumatoris comparats'].between(11, 15)
Percentatge_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.mean() * 100
num_observacions_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.sum()

print(f"Percentatge de highr_AMB_PA_MECVV: {Percentatge_highr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA_MECVV: {num_observacions_highr_AMB_PA_MECVV}")

##### SENSE_PA

In [ ]:
lowr_SENSE_PA = SENSE_PA['EMINA sumatoris comparats'].between(0, 5)
Percentatge_lowr_SENSE_PA = lowr_SENSE_PA.mean() * 100
num_observacions_lowr_SENSE_PA = lowr_SENSE_PA.sum()

print(f"Percentatge de lowr_SENSE_PA: {Percentatge_lowr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en lowr_SENSE_PA: {num_observacions_lowr_SENSE_PA}")

In [ ]:
modr_SENSE_PA = SENSE_PA['EMINA sumatoris comparats'].between(6, 10)
Percentatge_modr_SENSE_PA = modr_SENSE_PA.mean() * 100
num_observacions_modr_SENSE_PA = modr_SENSE_PA.sum()

print(f"Percentatge de modr_SENSE_PA: {Percentatge_modr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modr_SENSE_PA: {num_observacions_modr_SENSE_PA}")

In [ ]:
highr_SENSE_PA = SENSE_PA['EMINA sumatoris comparats'].between(11, 15)
Percentatge_highr_SENSE_PA = highr_SENSE_PA.mean() * 100
num_observacions_highr_SENSE_PA = highr_SENSE_PA.sum()

print(f"Percentatge de highr_SENSE_PA: {Percentatge_highr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en highr_SENSE_PA: {num_observacions_highr_SENSE_PA}")

##### TOTAL

In [ ]:
lowr_TOTAL = TOTAL['EMINA sumatoris comparats'].between(0, 5)
Percentatge_lowr_TOTAL = lowr_TOTAL.mean() * 100
num_observacions_lowr_TOTAL = lowr_TOTAL.sum()

print(f"Percentatge de lowr_TOTAL: {Percentatge_lowr_TOTAL:.2f}%")
print(f"Nombre d'observacions en lowr_TOTAL: {num_observacions_lowr_TOTAL}")

In [ ]:
modr_TOTAL = TOTAL['EMINA sumatoris comparats'].between(6, 10)
Percentatge_modr_TOTAL = modr_TOTAL.mean() * 100
num_observacions_modr_TOTAL = modr_TOTAL.sum()

print(f"Percentatge de modr_TOTAL: {Percentatge_modr_TOTAL:.2f}%")
print(f"Nombre d'observacions en modr_TOTAL: {num_observacions_modr_TOTAL}")

In [ ]:
highr_TOTAL = TOTAL['EMINA sumatoris comparats'].between(11, 15)
Percentatge_highr_TOTAL = highr_TOTAL.mean() * 100
num_observacions_highr_TOTAL = highr_TOTAL.sum()

print(f"Percentatge de highr_TOTAL: {Percentatge_highr_TOTAL:.2f}%")
print(f"Nombre d'observacions en highr_TOTAL: {num_observacions_highr_TOTAL}")

#### P-valor

##### Risc baix

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['EMINA sumatoris comparats'].between(0,5), columns=AMB_PA_MECVV['EMINA sumatoris comparats'].between(0,5))

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['EMINA sumatoris comparats'].between(0,5), columns=SENSE_PA['EMINA sumatoris comparats'].between(0,5))

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['EMINA sumatoris comparats'].between(0,5), columns=SENSE_PA['EMINA sumatoris comparats'].between(0,5))

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['EMINA sumatoris comparats'].between(0,5), columns=TOTAL['EMINA sumatoris comparats'].between(0,5))

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['EMINA sumatoris comparats'].between(0,5), columns=TOTAL['EMINA sumatoris comparats'].between(0,5))

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['EMINA sumatoris comparats'].between(0,5), columns=TOTAL['EMINA sumatoris comparats'].between(0,5))

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en SENSE_PA i TOTAL.")

##### Risc moderat

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['EMINA sumatoris comparats'].between(6,10), columns=AMB_PA_MECVV['EMINA sumatoris comparats'].between(6,10))

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['EMINA sumatoris comparats'].between(6,10), columns=SENSE_PA['EMINA sumatoris comparats'].between(6,10))

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['EMINA sumatoris comparats'].between(6,10), columns=SENSE_PA['EMINA sumatoris comparats'].between(6,10))

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['EMINA sumatoris comparats'].between(6,10), columns=TOTAL['EMINA sumatoris comparats'].between(6,10))

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['EMINA sumatoris comparats'].between(6,10), columns=TOTAL['EMINA sumatoris comparats'].between(6,10))

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['EMINA sumatoris comparats'].between(6,10), columns=TOTAL['EMINA sumatoris comparats'].between(6,10))

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en SENSE_PA i TOTAL.")

##### Risc alt

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['EMINA sumatoris comparats'].between(11,15), columns=AMB_PA_MECVV['EMINA sumatoris comparats'].between(11,15))

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['EMINA sumatoris comparats'].between(11,15), columns=SENSE_PA['EMINA sumatoris comparats'].between(11,15))

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['EMINA sumatoris comparats'].between(11,15), columns=SENSE_PA['EMINA sumatoris comparats'].between(11,15))

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['EMINA sumatoris comparats'].between(11,15), columns=TOTAL['EMINA sumatoris comparats'].between(11,15))

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['EMINA sumatoris comparats'].between(11,15), columns=TOTAL['EMINA sumatoris comparats'].between(11,15))

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['EMINA sumatoris comparats'].between(11,15), columns=TOTAL['EMINA sumatoris comparats'].between(11,15))

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'EMINA en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'EMINA en SENSE_PA i TOTAL.")

## Index de la canadenca (escala neurològica canadenca)

### Mitjana

In [ ]:
Mitjana_canad = AMB_PA['Canadenca sumatoris comparats'].mean()
desviacio_estandard_canad = AMB_PA['Canadenca sumatoris comparats'].std()

resultat = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultat)

In [ ]:
Mitjana_canad = AMB_PA_MECVV['Canadenca sumatoris comparats'].mean()
desviacio_estandard_canad = AMB_PA_MECVV['Canadenca sumatoris comparats'].std()

resultat = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultat)

In [ ]:
Mitjana_canad = SENSE_PA['Canadenca sumatoris comparats'].mean()
desviacio_estandard_canad = SENSE_PA['Canadenca sumatoris comparats'].std()

resultat = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultat)

In [ ]:
Mitjana_canad = TOTAL['Canadenca sumatoris comparats'].mean()
desviacio_estandard_canad = TOTAL['Canadenca sumatoris comparats'].std()

resultat = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
stat, p_value = shapiro(AMB_PA['Canadenca sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(AMB_PA_MECVV['Canadenca sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(SENSE_PA['Canadenca sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(TOTAL['Canadenca sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Tots els grups segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'], AMB_PA_MECVV['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'], SENSE_PA['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca sumatoris comparats'], SENSE_PA['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'], TOTAL['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Canadenca sumatoris comparats'], TOTAL['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Canadenca sumatoris comparats'], TOTAL['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Valors canadenca

##### AMB_PA 

In [ ]:
mild_AMB_PA = AMB_PA['Canadenca sumatoris comparats'].between(10.5, 11.5)
Percentatge_mild_AMB_PA = mild_AMB_PA.mean() * 100
num_observacions_mild_AMB_PA = mild_AMB_PA.sum()

print(f"Percentatge de mild_AMB_PA: {Percentatge_mild_AMB_PA:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA: {num_observacions_mild_AMB_PA}")

In [ ]:
modneu_AMB_PA = AMB_PA['Canadenca sumatoris comparats'].between(5,10)
Percentatge_modneu_AMB_PA = modneu_AMB_PA.mean() * 100
num_observacions_modneu_AMB_PA = modneu_AMB_PA.sum()

print(f"Percentatge de modneu_AMB_PA: {Percentatge_modneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA: {num_observacions_modneu_AMB_PA}")

In [ ]:
sevneu_AMB_PA = AMB_PA['Canadenca sumatoris comparats'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA = sevneu_AMB_PA.mean() * 100
num_observacions_sevneu_AMB_PA = sevneu_AMB_PA.sum()

print(f"Percentatge de sevneu_AMB_PA: {Percentatge_sevneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA: {num_observacions_sevneu_AMB_PA}")

##### AMB_PA_MECVV

In [ ]:
mild_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca sumatoris comparats'].between(10.5, 11.5)
Percentatge_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.mean() * 100
num_observacions_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.sum()

print(f"Percentatge de mild_AMB_PA_MECVV: {Percentatge_mild_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA_MECVV: {num_observacions_mild_AMB_PA_MECVV}")

In [ ]:
modneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca sumatoris comparats'].between(5,10)
Percentatge_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.mean() * 100
num_observacions_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.sum()

print(f"Percentatge de modneu_AMB_PA_MECVV: {Percentatge_modneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA_MECVV: {num_observacions_modneu_AMB_PA_MECVV}")

In [ ]:
sevneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca sumatoris comparats'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.mean() * 100
num_observacions_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.sum()

print(f"Percentatge de sevneu_AMB_PA_MECVV: {Percentatge_sevneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA_MECVV: {num_observacions_sevneu_AMB_PA_MECVV}")

##### SENSE_PA

In [ ]:
mild_SENSE_PA = SENSE_PA['Canadenca sumatoris comparats'].between(10.5, 11.5)
Percentatge_mild_SENSE_PA = mild_SENSE_PA.mean() * 100
num_observacions_mild_SENSE_PA = mild_SENSE_PA.sum()

print(f"Percentatge de mild_SENSE_PA: {Percentatge_mild_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en mild_SENSE_PA: {num_observacions_mild_SENSE_PA}")

In [ ]:
modneu_SENSE_PA = SENSE_PA['Canadenca sumatoris comparats'].between(5,10)
Percentatge_modneu_SENSE_PA = modneu_SENSE_PA.mean() * 100
num_observacions_modneu_SENSE_PA = modneu_SENSE_PA.sum()

print(f"Percentatge de modneu_SENSE_PA: {Percentatge_modneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modneu_SENSE_PA: {num_observacions_modneu_SENSE_PA}")

In [ ]:
sevneu_SENSE_PA = SENSE_PA['Canadenca sumatoris comparats'].between(1.5, 4.5)
Percentatge_sevneu_SENSE_PA = sevneu_SENSE_PA.mean() * 100
num_observacions_sevneu_SENSE_PA = sevneu_SENSE_PA.sum()

print(f"Percentatge de sevneu_SENSE_PA: {Percentatge_sevneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_SENSE_PA: {num_observacions_sevneu_SENSE_PA}")

##### TOTAL

In [ ]:
mild_TOTAL = TOTAL['Canadenca sumatoris comparats'].between(10.5, 11.5)
Percentatge_mild_TOTAL = mild_TOTAL.mean() * 100
num_observacions_mild_TOTAL = mild_TOTAL.sum()

print(f"Percentatge de mild_TOTAL: {Percentatge_mild_TOTAL:.2f}%")
print(f"Nombre d'observacions en mild_TOTAL: {num_observacions_mild_TOTAL}")

In [ ]:
modneu_TOTAL = TOTAL['Canadenca sumatoris comparats'].between(5,10)
Percentatge_modneu_TOTAL = modneu_TOTAL.mean() * 100
num_observacions_modneu_TOTAL = modneu_TOTAL.sum()

print(f"Percentatge de modneu_TOTAL: {Percentatge_modneu_TOTAL:.2f}%")
print(f"Nombre d'observacions en modneu_TOTAL: {num_observacions_modneu_TOTAL}")

In [ ]:
sevneu_TOTAL = TOTAL['Canadenca sumatoris comparats'].between(1.5, 4.5)
Percentatge_sevneu_TOTAL = sevneu_TOTAL.mean() * 100
num_observacions_sevneu_TOTAL = sevneu_TOTAL.sum()

print(f"Percentatge de sevneu_TOTAL: {Percentatge_sevneu_TOTAL:.2f}%")
print(f"Nombre d'observacions en sevneu_TOTAL: {num_observacions_sevneu_TOTAL}")

#### P-valor

##### Dèficit neurològic lleuger (mild)

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Canadenca sumatoris comparats'].between(10.5,11.5), columns=AMB_PA_MECVV['Canadenca sumatoris comparats'].between(10.5,11.5))

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
   print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i AMB_PA_MECVV.")
else:
   print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i AMB_PA_MECVV.")


In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Canadenca sumatoris comparats'].between(10.5,11.5), columns=SENSE_PA['Canadenca sumatoris comparats'].between(10.5,11.5))

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
   print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i SENSE_PA.")
else:
   print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Canadenca sumatoris comparats'].between(10.5,11.5), columns=SENSE_PA['Canadenca sumatoris comparats'].between(10.5,11.5))

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
   print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i SENSE_PA.")
else:
   print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Canadenca sumatoris comparats'].between(10.5,11.5), columns=TOTAL['Canadenca sumatoris comparats'].between(10.5,11.5))

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Canadenca sumatoris comparats'].between(10.5,11.5), columns=TOTAL['Canadenca sumatoris comparats'].between(10.5,11.5))

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Canadenca sumatoris comparats'].between(10.5,11.5), columns=TOTAL['Canadenca sumatoris comparats'].between(10.5,11.5))

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de canadenca en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de canadenca en SENSE_PA i TOTAL.")

##### Dèficit neurològic moderat

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Canadenca sumatoris comparats'].between(5,10), columns=AMB_PA_MECVV['Canadenca sumatoris comparats'].between(5,10))

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
   print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i AMB_PA_MECVV.")
else:
   print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Canadenca sumatoris comparats'].between(5,10), columns=SENSE_PA['Canadenca sumatoris comparats'].between(5,10))

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
   print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i SENSE_PA.")
else:
   print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Canadenca sumatoris comparats'].between(5,10), columns=SENSE_PA['Canadenca sumatoris comparats'].between(5,10))

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
   print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i SENSE_PA.")
else:
   print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Canadenca sumatoris comparats'].between(5,10), columns=TOTAL['Canadenca sumatoris comparats'].between(5,10))

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Canadenca sumatoris comparats'].between(5,10), columns=TOTAL['Canadenca sumatoris comparats'].between(5,10))

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Canadenca sumatoris comparats'].between(5,10), columns=TOTAL['Canadenca sumatoris comparats'].between(5,10))

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de canadenca en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de canadenca en SENSE_PA i TOTAL.")

##### Dèficit neurològic greu (sever)

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Canadenca sumatoris comparats'].between(1.5, 4.5), columns=AMB_PA_MECVV['Canadenca sumatoris comparats'].between(1.5, 4.5))

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
   print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i AMB_PA_MECVV.")
else:
   print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Canadenca sumatoris comparats'].between(1.5, 4.5), columns=SENSE_PA['Canadenca sumatoris comparats'].between(1.5, 4.5))

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
   print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i SENSE_PA.")
else:
   print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Canadenca sumatoris comparats'].between(1.5, 4.5), columns=SENSE_PA['Canadenca sumatoris comparats'].between(1.5, 4.5))

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
   print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i SENSE_PA.")
else:
   print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Canadenca sumatoris comparats'].between(1.5, 4.5), columns=TOTAL['Canadenca sumatoris comparats'].between(1.5, 4.5))

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Canadenca sumatoris comparats'].between(1.5, 4.5), columns=TOTAL['Canadenca sumatoris comparats'].between(1.5, 4.5))

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de canadenca en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Canadenca sumatoris comparats'].between(1.5, 4.5), columns=TOTAL['Canadenca sumatoris comparats'].between(1.5, 4.5))

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de canadenca en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de canadenca en SENSE_PA i TOTAL.")

## Infeccions respiratòries

##### AMB_PA 

In [ ]:
mainresp_AMB_PA= (AMB_PA['Infeccions respiratòries principals diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA:.2f}%")

In [ ]:
LRTI_AMB_PA= (AMB_PA['LRTI diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA:.2f}%")

In [ ]:
COPD_AMB_PA= (AMB_PA['Exacerbacions de COPD diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA:.2f}%")

In [ ]:
fibro_AMB_PA= (AMB_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA:.2f}%")

##### AMB_PA_MECVV

In [ ]:
mainresp_AMB_PA_MECVV= (AMB_PA_MECVV['Infeccions respiratòries principals diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA_MECVV:.2f}%")

In [ ]:
LRTI_AMB_PA_MECVV= (AMB_PA_MECVV['LRTI diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA_MECVV:.2f}%")

In [ ]:
COPD_AMB_PA_MECVV = (AMB_PA_MECVV['Exacerbacions de COPD diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA_MECVV:.2f}%")

In [ ]:
fibro_AMB_PA_MECVV = (AMB_PA_MECVV['Fibrosi pulmonar i fibrotòrax diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA_MECVV:.2f}%")

##### SENSE_PA

In [ ]:
mainresp_SENSE_PA= (SENSE_PA['Infeccions respiratòries principals diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_SENSE_PA:.2f}%")

In [ ]:
LRTI_SENSE_PA= (SENSE_PA['LRTI diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_SENSE_PA:.2f}%")

In [ ]:
COPD_SENSE_PA= (SENSE_PA['Exacerbacions de COPD diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_SENSE_PA:.2f}%")

In [ ]:
fibro_SENSE_PA= (SENSE_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_SENSE_PA:.2f}%")

#### Comparació distributiva

##### Normalitat i p-valor

No cal calcular la normalitat ja que totes són variables categòriques

##### Infeccions respiratòries principals

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Infeccions respiratòries principals diagnosticades'], columns=AMB_PA_MECVV['Infeccions respiratòries principals diagnosticades'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Infeccions respiratòries principals diagnosticades'], columns=SENSE_PA['Infeccions respiratòries principals diagnosticades'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Infeccions respiratòries principals diagnosticades'], columns=SENSE_PA['Infeccions respiratòries principals diagnosticades'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Infeccions respiratòries principals diagnosticades'], columns=TOTAL['Infeccions respiratòries principals diagnosticades'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_14)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Infeccions respiratòries principals diagnosticades'], columns=TOTAL['Infeccions respiratòries principals diagnosticades'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Infeccions respiratòries principals diagnosticades'], columns=TOTAL['Infeccions respiratòries principals diagnosticades'])

chi2_34, p_value_34, dof_4, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre AMB_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en SENSE_PA i TOTAL.")

##### LRTI

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['LRTI diagnosticada'], columns=AMB_PA_MECVV['LRTI diagnosticada'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['LRTI diagnosticada'], columns=SENSE_PA['LRTI diagnosticada'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['LRTI diagnosticada'], columns=SENSE_PA['LRTI diagnosticada'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['LRTI diagnosticada'], columns=TOTAL['LRTI diagnosticada'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['LRTI diagnosticada'], columns=TOTAL['LRTI diagnosticada'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['LRTI diagnosticada'], columns=TOTAL['LRTI diagnosticada'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SNESE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en SENSE_PA i TOTAL.")

##### Exacerbacions de COPD

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Exacerbacions de COPD diagnosticades'], columns=AMB_PA_MECVV['Exacerbacions de COPD diagnosticades'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i AMB_PA_MECV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Exacerbacions de COPD diagnosticades'], columns=SENSE_PA['Exacerbacions de COPD diagnosticades'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Exacerbacions de COPD diagnosticades'], columns=SENSE_PA['Exacerbacions de COPD diagnosticades'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Exacerbacions de COPD diagnosticades'], columns=TOTAL['Exacerbacions de COPD diagnosticades'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Exacerbacions de COPD diagnosticades'], columns=TOTAL['Exacerbacions de COPD diagnosticades'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Exacerbacions de COPD diagnosticades'], columns=TOTAL['Exacerbacions de COPD diagnosticades'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en SENSE_PA i TOTAL.")

##### Fibrosi pulmonar i fibrotòrax

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=AMB_PA_MECVV['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=SENSE_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=SENSE_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA_MECVV i SENSE_PA.")

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=TOTAL['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=TOTAL['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=TOTAL['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en SENSE_PA i TOTAL.")

## Resultats de laboratori

Les columnes generades a partir d'aquests resultats contenen valors tipus objecte, per tant, primer els transformaré a tipus numèric per poder realitzar les operacions que neccessitaré fer.

### Albúmina

In [ ]:
AMB_PA.loc[:, 'Albúmina'] = pd.to_numeric(AMB_PA['Albúmina'], errors='coerce').dropna()

Mitjana_alb = AMB_PA['Albúmina'].mean()
desviacio_estandard_alb = AMB_PA['Albúmina'].std()

resultat = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de l'albúmina:", resultat)

In [ ]:
AMB_PA_MECVV.loc[:, 'Albúmina'] = pd.to_numeric(AMB_PA_MECVV['Albúmina'], errors='coerce').dropna()

Mitjana_alb = AMB_PA_MECVV['Albúmina'].mean()
desviacio_estandard_alb = AMB_PA_MECVV['Albúmina'].std()

resultat = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de l'albúmina:", resultat)

In [ ]:
SENSE_PA.loc[:, 'Albúmina'] = pd.to_numeric(SENSE_PA['Albúmina'], errors='coerce').dropna()

Mitjana_alb = SENSE_PA['Albúmina'].mean()
desviacio_estandard_alb = SENSE_PA['Albúmina'].std()

resultat = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de l'albúmina:", resultat)

In [ ]:
TOTAL.loc[:, 'Albúmina'] = pd.to_numeric(TOTAL['Albúmina'], errors='coerce').dropna()

Mitjana_alb = TOTAL['Albúmina'].mean()
desviacio_estandard_alb = TOTAL['Albúmina'].std()

resultat = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de l'albúmina:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
albpa = AMB_PA['Albúmina']
stat, p_value = shapiro(albpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
albAMBpamecvv = AMB_PA_MECVV['Albúmina']
stat, p_value = shapiro(albAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
albsensepa= SENSE_PA['Albúmina']
stat, p_value = shapiro(albsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
albtotal = TOTAL['Albúmina']
stat, p_value = shapiro(albtotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Els 4 grups segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Albúmina'], AMB_PA_MECVV['Albúmina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Albúmina'], SENSE_PA['Albúmina'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Albúmina'], SENSE_PA['Albúmina'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Albúmina'], TOTAL['Albúmina'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Albúmina'], TOTAL['Albúmina'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_14 = ttest_ind(SENSE_PA['Albúmina'], TOTAL['Albúmina'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Proteïnes totals

In [ ]:
AMB_PA.loc[:, 'Proteïnes totals'] = pd.to_numeric(AMB_PA['Proteïnes totals'], errors='coerce')

Mitjana_prot = AMB_PA['Proteïnes totals'].mean()
desviacio_estandard_prot = AMB_PA['Proteïnes totals'].std()

resultat = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de les proteïnes totals:", resultat)

In [ ]:
AMB_PA_MECVV.loc[:, 'Proteïnes totals'] = pd.to_numeric(AMB_PA_MECVV['Proteïnes totals'], errors='coerce')

Mitjana_prot = AMB_PA_MECVV['Proteïnes totals'].mean()
desviacio_estandard_prot = AMB_PA_MECVV['Proteïnes totals'].std()

resultat = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de les proteïnes totals:", resultat)

In [ ]:
SENSE_PA.loc[:, 'Proteïnes totals'] = pd.to_numeric(SENSE_PA['Proteïnes totals'], errors='coerce')

Mitjana_prot = SENSE_PA['Proteïnes totals'].mean()
desviacio_estandard_prot = SENSE_PA['Proteïnes totals'].std()

resultat = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de les proteïnes totals:", resultat)

In [ ]:
TOTAL.loc[:, 'Proteïnes totals'] = pd.to_numeric(TOTAL['Proteïnes totals'], errors='coerce')

Mitjana_prot = TOTAL['Proteïnes totals'].mean()
desviacio_estandard_prot = TOTAL['Proteïnes totals'].std()

resultat = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de les proteïnes totals:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
protAMBpa=AMB_PA['Proteïnes totals']
stat, p_value = shapiro(protAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
protAMBpamecvv=AMB_PA_MECVV['Proteïnes totals']
stat, p_value = shapiro(protAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
protsensepa=SENSE_PA['Proteïnes totals']
stat, p_value = shapiro(protsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
prottotal=TOTAL['Proteïnes totals']
stat, p_value = shapiro(prottotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Els 4 grups segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Proteïnes totals'], AMB_PA_MECVV['Proteïnes totals'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Proteïnes totals'], SENSE_PA['Proteïnes totals'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Proteïnes totals'], SENSE_PA['Proteïnes totals'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Proteïnes totals'], TOTAL['Proteïnes totals'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Proteïnes totals'], TOTAL['Proteïnes totals'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Proteïnes totals'], TOTAL['Proteïnes totals'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Hemoglobina

In [ ]:
AMB_PA.loc[:, 'Hb'] = pd.to_numeric(AMB_PA['Hb'], errors='coerce')

Mitjana_hb = AMB_PA['Hb'].mean()
desviacio_estandard_hb = AMB_PA['Hb'].std()

resultat = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de l'hemoglobina:", resultat)

In [ ]:
AMB_PA_MECVV.loc[:, 'Hb'] = pd.to_numeric(AMB_PA_MECVV['Hb'], errors='coerce')

Mitjana_hb = AMB_PA_MECVV['Hb'].mean()
desviacio_estandard_hb = AMB_PA_MECVV['Hb'].std()

resultat = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de l'hemoglobina:", resultat)

In [ ]:
SENSE_PA.loc[:, 'Hb'] = pd.to_numeric(SENSE_PA['Hb'], errors='coerce')

Mitjana_hb = SENSE_PA['Hb'].mean()
desviacio_estandard_hb = SENSE_PA['Hb'].std()

resultat = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de l'hemoglobina:", resultat)

In [ ]:
TOTAL.loc[:, 'Hb'] = pd.to_numeric(TOTAL['Hb'], errors='coerce')

Mitjana_hb = TOTAL['Hb'].mean()
desviacio_estandard_hb = TOTAL['Hb'].std()

resultat = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
hbAMBpa=AMB_PA['Hb'].dropna()
stat, p_value = shapiro(hbAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
hbAMBpamecvv=AMB_PA_MECVV['Hb'].dropna()
stat, p_value = shapiro(hbAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
hbsensepa=SENSE_PA['Hb'].dropna()
stat, p_value = shapiro(hbsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
hbtotal=TOTAL['Hb'].dropna()
stat, p_value = shapiro(hbtotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Els 4 grups segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Hb'], AMB_PA_MECVV['Hb'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Hb'], TOTAL['Hb'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Hb'], TOTAL['Hb'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Hb'], TOTAL['Hb'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Colesterol 

In [ ]:
AMB_PA.loc[:, 'Colesterol total'] = pd.to_numeric(AMB_PA['Colesterol total'], errors='coerce')

Mitjana_col = AMB_PA['Colesterol total'].mean()
desviacio_estandard_col = AMB_PA['Colesterol total'].std()

resultat = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultat)

In [ ]:
AMB_PA_MECVV.loc[:, 'Colesterol total'] = pd.to_numeric(AMB_PA_MECVV['Colesterol total'], errors='coerce')

Mitjana_col = AMB_PA_MECVV['Colesterol total'].mean()
desviacio_estandard_col = AMB_PA_MECVV['Colesterol total'].std()

resultat = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultat)

In [ ]:
SENSE_PA.loc[:, 'Colesterol total'] = pd.to_numeric(SENSE_PA['Colesterol total'], errors='coerce')

Mitjana_col = SENSE_PA['Colesterol total'].mean()
desviacio_estandard_col = SENSE_PA['Colesterol total'].std()

resultat = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultat)

In [ ]:
TOTAL.loc[:, 'Colesterol total'] = pd.to_numeric(TOTAL['Colesterol total'], errors='coerce')

Mitjana_col = TOTAL['Colesterol total'].mean()
desviacio_estandard_col = TOTAL['Colesterol total'].std()

resultat = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor



In [ ]:
colAMBpa=AMB_PA['Colesterol total'].dropna()
stat, p_value = shapiro(colAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
colAMBpamecvv=AMB_PA_MECVV['Colesterol total'].dropna()
stat, p_value = shapiro(colAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
colsensepa=SENSE_PA['Colesterol total'].dropna()
stat, p_value = shapiro(colsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
coltotal=TOTAL['Colesterol total'].dropna()
stat, p_value = shapiro(coltotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Els 4 grups segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Colesterol total'], AMB_PA_MECVV['Colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Colesterol total'], SENSE_PA['Colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Colesterol total'], SENSE_PA['Colesterol total'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Colesterol total'], TOTAL['Colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Colesterol total'], TOTAL['Colesterol total'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Colesterol total'], TOTAL['Colesterol total'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Leucòcits

In [ ]:
AMB_PA.loc[:, 'Leucos'] = pd.to_numeric(AMB_PA['Leucos'], errors='coerce')

Mitjana_leu = AMB_PA['Leucos'].mean()
desviacio_estandard_leu = AMB_PA['Leucos'].std()

resultat = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultat)

In [ ]:
AMB_PA_MECVV.loc[:, 'Leucos'] = pd.to_numeric(AMB_PA_MECVV['Leucos'], errors='coerce')

Mitjana_leu = AMB_PA_MECVV['Leucos'].mean()
desviacio_estandard_leu = AMB_PA_MECVV['Leucos'].std()

resultat = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultat)

In [ ]:
SENSE_PA.loc[:, 'Leucos'] = pd.to_numeric(SENSE_PA['Leucos'], errors='coerce')

Mitjana_leu = SENSE_PA['Leucos'].mean()
desviacio_estandard_leu = SENSE_PA['Leucos'].std()

resultat = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultat)

In [ ]:
TOTAL.loc[:, 'Leucos'] = pd.to_numeric(TOTAL['Leucos'], errors='coerce')

Mitjana_leu = TOTAL['Leucos'].mean()
desviacio_estandard_leu = TOTAL['Leucos'].std()

resultat = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
leuambpa=AMB_PA['Leucos'].dropna()
stat, p_value = shapiro(leuambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
leuambpamecvv=AMB_PA_MECVV['Leucos'].dropna()
stat, p_value = shapiro(leuambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
leusensepa=SENSE_PA['Leucos'].dropna()
stat, p_value = shapiro(leusensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
leutotal=TOTAL['Leucos'].dropna()
stat, p_value = shapiro(leutotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Els 4 grups segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Leucos'], AMB_PA_MECVV['Leucos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Leucos'], SENSE_PA['Leucos'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Leucos'], SENSE_PA['Leucos'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Leucos'], TOTAL['Leucos'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Leucos'], TOTAL['Leucos'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Leucos'], TOTAL['Leucos'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Limfòcits %

In [ ]:
AMB_PA.loc[:, 'Limfos'] = pd.to_numeric(AMB_PA['Limfos'], errors='coerce')

Mitjana_limf = AMB_PA['Limfos'].mean()
desviacio_estandard_limf = AMB_PA['Limfos'].std()

resultat = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultat)

In [ ]:
AMB_PA_MECVV.loc[:, 'Limfos'] = pd.to_numeric(AMB_PA_MECVV['Limfos'], errors='coerce')

Mitjana_limf = AMB_PA_MECVV['Limfos'].mean()
desviacio_estandard_limf = AMB_PA_MECVV['Limfos'].std()

resultat = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultat)

In [ ]:
SENSE_PA.loc[:, 'Limfos'] = pd.to_numeric(SENSE_PA['Limfos'], errors='coerce')

Mitjana_limf = SENSE_PA['Limfos'].mean()
desviacio_estandard_limf = SENSE_PA['Limfos'].std()

resultat = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultat)

In [ ]:
TOTAL.loc[:, 'Limfos'] = pd.to_numeric(TOTAL['Limfos'], errors='coerce')

Mitjana_limf = TOTAL['Limfos'].mean()
desviacio_estandard_limf = TOTAL['Limfos'].std()

resultat = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
limfambpa=AMB_PA['Limfos'].dropna()
stat, p_value = shapiro(limfambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
limfambpamecvv=AMB_PA_MECVV['Limfos'].dropna()
stat, p_value = shapiro(limfambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
limfsensepa=SENSE_PA['Limfos'].dropna()
stat, p_value = shapiro(limfsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
limftotal=TOTAL['Limfos'].dropna()
stat, p_value = shapiro(limftotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Tots els grups, menys el primer (AMB_PA), segueixen una distribució normal

In [ ]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['Limfos'], AMB_PA_MECVV['Limfos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['Limfos'], SENSE_PA['Limfos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Limfos'], SENSE_PA['Limfos'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
stat_14, p_val_14 = mannwhitneyu(AMB_PA['Limfos'], TOTAL['Limfos'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Limfos'], TOTAL['Limfos'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
stat_34, p_val_34 = ttest_ind(SENSE_PA['Limfos'], TOTAL['Limfos'], equal_var=False)
print("P-valor entre SNESE_PA i TOTAL:", p_val_14)

### Proteïna C Reactiva

In [ ]:
AMB_PA.loc[:, 'Prot C react'] = pd.to_numeric(AMB_PA['Prot C react'], errors='coerce')

Mitjana_protcr = AMB_PA['Prot C react'].mean()
desviacio_estandard_protcr = AMB_PA['Prot C react'].std()

resultat = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultat)

In [ ]:
AMB_PA_MECVV.loc[:, 'Prot C react'] = pd.to_numeric(AMB_PA_MECVV['Prot C react'], errors='coerce')

Mitjana_protcr = AMB_PA_MECVV['Prot C react'].mean()
desviacio_estandard_protcr = AMB_PA_MECVV['Prot C react'].std()

resultat = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultat)

In [ ]:
SENSE_PA.loc[:, 'Prot C react'] = pd.to_numeric(SENSE_PA['Prot C react'], errors='coerce')

Mitjana_protcr = SENSE_PA['Prot C react'].mean()
desviacio_estandard_protcr = SENSE_PA['Prot C react'].std()

resultat = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultat)

In [ ]:
TOTAL.loc[:, 'Prot C react'] = pd.to_numeric(TOTAL['Prot C react'], errors='coerce')

Mitjana_protcr = TOTAL['Prot C react'].mean()
desviacio_estandard_protcr = TOTAL['Prot C react'].std()

resultat = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
pcrambpa=AMB_PA['Prot C react'].dropna()
stat, p_value = shapiro(pcrambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
pcrambpamecvv=AMB_PA_MECVV['Prot C react'].dropna()
stat, p_value = shapiro(pcrambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
pcrsensepa=SENSE_PA['Prot C react'].dropna()
stat, p_value = shapiro(pcrsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
pcrtotal=TOTAL['Prot C react'].dropna()
stat, p_value = shapiro(pcrtotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Tots els grups, menys el quart (TOTAL), segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Prot C react'], AMB_PA_MECVV['Prot C react'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Prot C react'], SENSE_PA['Prot C react'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Prot C react'], SENSE_PA['Prot C react'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = mannwhitneyu(AMB_PA['Prot C react'], TOTAL['Prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = mannwhitneyu(AMB_PA_MECVV['Prot C react'], TOTAL['Prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_34 = mannwhitneyu(SENSE_PA['Prot C react'], TOTAL['Prot C react'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Urea 

In [ ]:
AMB_PA.loc[:, 'Urea'] = pd.to_numeric(AMB_PA['Urea'], errors='coerce')

Mitjana_Urea = AMB_PA['Urea'].mean()
desviacio_estandard_Urea = AMB_PA['Urea'].std()

resultat = f"{Mitjana_Urea:.2f} ± {desviacio_estandard_Urea:.2f}"
print("Mitjana ± Desviació Estàndard de la Urea:", resultat)

In [ ]:
AMB_PA_MECVV.loc[:, 'Urea'] = pd.to_numeric(AMB_PA_MECVV['Urea'], errors='coerce')

Mitjana_Urea = AMB_PA_MECVV['Urea'].mean()
desviacio_estandard_Urea = AMB_PA_MECVV['Urea'].std()

resultat = f"{Mitjana_Urea:.2f} ± {desviacio_estandard_Urea:.2f}"
print("Mitjana ± Desviació Estàndard de la Urea:", resultat)

In [ ]:
SENSE_PA.loc[:, 'Urea'] = pd.to_numeric(SENSE_PA['Urea'], errors='coerce')

Mitjana_Urea = SENSE_PA['Urea'].mean()
desviacio_estandard_Urea = SENSE_PA['Urea'].std()

resultat = f"{Mitjana_Urea:.2f} ± {desviacio_estandard_Urea:.2f}"
print("Mitjana ± Desviació Estàndard de la Urea:", resultat)

In [ ]:
TOTAL.loc[:, 'Urea'] = pd.to_numeric(TOTAL['Urea'], errors='coerce')

Mitjana_Urea = TOTAL['Urea'].mean()
desviacio_estandard_Urea = TOTAL['Urea'].std()

resultat = f"{Mitjana_Urea:.2f} ± {desviacio_estandard_Urea:.2f}"
print("Mitjana ± Desviació Estàndard de la Urea:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
uambpa=AMB_PA['Urea'].dropna()
stat, p_value = shapiro(uambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
uambpamecvv=AMB_PA_MECVV['Urea'].dropna()
stat, p_value = shapiro(uambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
usensepa=SENSE_PA['Urea'].dropna()
stat, p_value = shapiro(usensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
utotal=TOTAL['Urea'].dropna()
stat, p_value = shapiro(utotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Tots els grups, menys el quart (TOTAL), segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Urea'], AMB_PA_MECVV['Urea'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Urea'], SENSE_PA['Urea'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Urea'], SENSE_PA['Urea'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = mannwhitneyu(AMB_PA['Urea'], TOTAL['Urea'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = mannwhitneyu(AMB_PA_MECVV['Urea'], TOTAL['Urea'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_34 = mannwhitneyu(SENSE_PA['Urea'], TOTAL['Urea'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Filtrat glomerular CDK-EPI

In [ ]:
AMB_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = AMB_PA['FGE CDK-EPI'].mean()
desviacio_estandard_fg = AMB_PA['FGE CDK-EPI'].std()

resultat = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultat)

In [ ]:
AMB_PA_MECVV.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA_MECVV['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = AMB_PA_MECVV['FGE CDK-EPI'].mean()
desviacio_estandard_fg = AMB_PA_MECVV['FGE CDK-EPI'].std()

resultat = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultat)

In [ ]:
SENSE_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(SENSE_PA['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = SENSE_PA['FGE CDK-EPI'].mean()
desviacio_estandard_fg = SENSE_PA['FGE CDK-EPI'].std()

resultat = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultat)

In [ ]:
TOTAL.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(TOTAL['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = TOTAL['FGE CDK-EPI'].mean()
desviacio_estandard_fg = TOTAL['FGE CDK-EPI'].std()

resultat = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
fgepiambpa=AMB_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
fgepiambpamecvv=AMB_PA_MECVV['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
fgepisensepa=SENSE_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepisensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
fgepitotal=TOTAL['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepitotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Tots els grups, menys el primer (AMB_PA), segueixen una distribució normal

In [ ]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], AMB_PA_MECVV['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

In [ ]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

In [ ]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

In [ ]:
t_stat_14, p_val_14 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], TOTAL['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['FGE CDK-EPI'], TOTAL['FGE CDK-EPI'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

In [ ]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['FGE CDK-EPI'], TOTAL['FGE CDK-EPI'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

### Filtrat glomerular MDRD

In [ ]:
AMB_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA['FGE MDRD'], errors='coerce')

Mitjana_fgmd = AMB_PA['FGE MDRD'].mean()
desviacio_estandard_fgmd = AMB_PA['FGE MDRD'].std()

resultat = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultat)

In [ ]:
AMB_PA_MECVV.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA_MECVV['FGE MDRD'], errors='coerce')

Mitjana_fgmd = AMB_PA_MECVV['FGE MDRD'].mean()
desviacio_estandard_fgmd = AMB_PA_MECVV['FGE MDRD'].std()

resultat = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultat)

In [ ]:
SENSE_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(SENSE_PA['FGE MDRD'], errors='coerce')

Mitjana_fgmd = SENSE_PA['FGE MDRD'].mean()
desviacio_estandard_fgmd = SENSE_PA['FGE MDRD'].std()

resultat = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultat)

In [ ]:
TOTAL.loc[:, 'FGE MDRD'] = pd.to_numeric(TOTAL['FGE MDRD'], errors='coerce')

Mitjana_fgmd = TOTAL['FGE MDRD'].mean()
desviacio_estandard_fgmd = TOTAL['FGE MDRD'].std()

resultat = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultat)

#### Comparació distributiva

##### Normalitat i p-valor

In [ ]:
stat, p_value = shapiro(AMB_PA['FGE MDRD'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")


In [ ]:
stat, p_value = shapiro(AMB_PA_MECVV['FGE MDRD'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(SENSE_PA['FGE MDRD'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(TOTAL['FGE MDRD'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

NORMALITAT: Els 4 grups segueixen una distribució normal

In [499]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['FGE MDRD'], AMB_PA_MECVV['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [500]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [501]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i AMB_PA_MECVV:", p_val_23)

P-valor entre AMB_PA_MECVV i AMB_PA_MECVV: nan


In [502]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['FGE MDRD'], TOTAL['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i TOTAl:", p_val_14)

P-valor entre AMB_PA i TOTAl: nan


In [503]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['FGE MDRD'], TOTAL['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAl:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAl: nan


In [504]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['FGE MDRD'], TOTAL['FGE MDRD'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan
